In [1]:
import os.path
from datetime import datetime as dt

import keras.utils.vis_utils
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from IPython.display import display, clear_output
from dateutil.relativedelta import relativedelta
from keras import Sequential, layers, losses, metrics, activations
from keras import callbacks
from keras.api.keras import optimizers

import file_helper

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATE_FORMAT = '%d.%m.%Y'

In [4]:
def convert_string_to_date(string_date: str) -> dt:
    return dt.strptime(string_date, DATE_FORMAT)

In [5]:
geochem_data = pd.read_excel(file_helper.get_geo_chem_file_path(), 'dubki_h_tau')

In [6]:
events_catalog_data = pd.read_csv(file_helper.get_events_catalog_file_path(), sep=';').iloc[10000:]
events_catalog_data['Date'] = pd.to_datetime(events_catalog_data['Date'], format=DATE_FORMAT)

In [7]:
DATE_TARGET_OFFSET = relativedelta(months=3)
DATE_TARGET_DURATION = relativedelta(months=1)
STARTING_EVENT_CLASS = 13

In [8]:
def split_data(date_target_offset: relativedelta, date_target_duration: relativedelta, starting_event_class) -> (pd.DataFrame, pd.DataFrame):
    preprocessed_data = pd.DataFrame(geochem_data).copy()

    # normalize_columns_in_dataframe(preprocessed_data, ['events', 'ascend', 'maximum', 'descend', 'minimum'])

    breakpoint_date = convert_string_to_date('31.12.2021') - date_target_offset - date_target_duration
    reserved_data = preprocessed_data[breakpoint_date < preprocessed_data['to date']].drop(['from date'], axis=1)
    preprocessed_data = preprocessed_data[preprocessed_data['to date'] <= breakpoint_date]
    preprocessed_data['target'] = preprocessed_data['to date'].map(lambda date: 1 if len(events_catalog_data[
        ((date + date_target_offset) <= events_catalog_data['Date'])
        & (events_catalog_data['Date'] <= (date + date_target_offset + date_target_duration))
        & (starting_event_class <= events_catalog_data['Class'])
    ]) > 0 else 0)
    preprocessed_data = preprocessed_data.drop(['from date', 'to date'], axis=1)

    return preprocessed_data, reserved_data

In [9]:
preprocessed_data, reserved_data = split_data(DATE_TARGET_OFFSET, DATE_TARGET_DURATION, STARTING_EVENT_CLASS)
preprocessed_data

,events,ascend,maximum,descend,minimum,target
0,41,2.021409,0.846639,1.065538,0.686655,0
1,42,1.973066,0.901546,1.040415,0.670327,0
2,42,1.973066,0.901546,1.040415,0.670327,0
3,42,1.973066,0.901546,1.040415,0.670327,0
4,42,1.973066,0.901546,1.040415,0.670327,0
...,...,...,...,...,...,...
5605,157,1.311948,1.076377,1.090581,0.693564,0
5606,157,1.317518,1.077638,1.090581,0.693564,0
5607,157,1.317518,1.087743,1.090581,0.687401,0
5608,156,1.282519,1.065515,1.097792,0.701725,0


In [10]:
# display(
#     preprocessed_data.tail(),
#     reserved_data.head()
# )
# geochem_data.head()

In [11]:
RANDOM_STATE = 42
TRAIN_FRAC = .8

In [12]:
def generate_train_test(data: pd.DataFrame, train: float = .8, seed: int = None):

    data_train = data.sample(frac=train, random_state=seed)
    data_test = data.drop(data_train.index)

    x_train, y_train = data_train.drop(['target'], axis=1), data_train['target']
    x_test, y_test = data_test.drop(['target'], axis=1), data_test['target']

    return (x_train, y_train), (x_test, y_test)

In [13]:
def model_builder(hp: kt.HyperParameters):
    # hp_date_target_offset_months = hp.Int('offset', 1, 12)
    # hp_date_target_duration_months = hp.Int('duration', 1, 12)

    hp_activation = hp.Choice('activation', values=['linear', 'relu', 'tanh', 'sigmoid'])
    hp_units = hp.Int('units', min_value=8, max_value=128, step=8)
    hp_activation_1 = hp.Choice('activation_1', values=['linear', 'relu', 'tanh', 'sigmoid'])
    hp_units_1 = hp.Int('units_1', min_value=8, max_value=128, step=8)
    hp_activation_2 = hp.Choice('activation_2', values=['linear', 'relu', 'tanh', 'sigmoid'])
    hp_units_2 = hp.Int('units_2', min_value=8, max_value=128, step=8)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    core_model = Sequential([
        layers.InputLayer((5,), name='input'),
        layers.Dense(units=hp_units, activation=hp_activation, name='dense_0'),
        layers.Dense(units=hp_units_1, activation=hp_activation_1, name='dense_1'),
        layers.Dense(units=hp_units_2, activation=hp_activation_2, name='dense_2'),
        layers.Dense(1, name='output')
    ], name='core_model')

    normalizer = keras.layers.Normalization(axis=1, name='normalization_layer')
    normalizer.adapt(preprocessed_data.drop('target', axis=1))

    inputs = keras.Input(shape=(5,))
    x = normalizer(inputs)
    outputs = core_model(x)

    model = keras.Model(inputs, outputs, name='model')
    model.compile(
        optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
        loss=losses.BinaryCrossentropy(from_logits=True),
        metrics=[metrics.BinaryAccuracy(name='accuracy')]
    )

    return model

In [14]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=200,
    factor=3,
    directory=os.path.join(file_helper.get_root_path(), f'data/hypermodel/date_3_1'),
    project_name='geo_analysis'
)

INFO:tensorflow:Reloading Oracle from existing project C:\Users\saaru\PycharmProjects\geo\data/hypermodel/date_3_1\geo_analysis\oracle.json
INFO:tensorflow:Reloading Tuner from C:\Users\saaru\PycharmProjects\geo\data/hypermodel/date_3_1\geo_analysis\tuner0.json


In [15]:
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [16]:
total_count = len(preprocessed_data)
neg_count = len(preprocessed_data[preprocessed_data['target'] == 0]['target'])
pos_count = len(preprocessed_data[preprocessed_data['target'] == 1]['target'])

weight_0 = (1 / neg_count) * (total_count / 2.0)
weight_1 = (1 / pos_count) * (total_count / 2.0)

print(weight_0, weight_1)
print(f'Negatives: {neg_count/total_count*100}%')
print(f'Positives: {pos_count/total_count*100}%')

0.5203116304952699 12.808219178082192
Negatives: 96.09625668449198%
Positives: 3.9037433155080214%


In [17]:
(x_train, y_train), (x_test, y_test) = generate_train_test(preprocessed_data, TRAIN_FRAC, RANDOM_STATE)

In [18]:
tuner.search(
    x_train, y_train,
    epochs=100,
    validation_split=.2,
    class_weight={0: weight_0, 1: weight_1},
    callbacks=[stop_early]
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

INFO:tensorflow:Oracle triggered exit


In [19]:
def search_best_model(seed: int=None):
    START_MONTH = 1
    END_MONTH = 12

    last_score = -1.
    best_score = -1.
    best_offset, best_duration = (-1, -1)

    for offset in range(START_MONTH, END_MONTH + 1):
        for duration in range(START_MONTH, END_MONTH + 1):
            clear_output(wait=True)
            print(f'{"Offset":10}: {offset}\n{"Duration":10}: {duration}')
            print(f'Best offset: {best_offset}\nBest duration: {best_duration}')
            print(f'Last score: {last_score}')
            print(f'Best score: {best_score}')

            date_target_offset = relativedelta(months=offset)
            date_target_duration = relativedelta(months=duration)

            stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=10)

            preprocessed_data, reserved_data = split_data(date_target_offset, date_target_duration, STARTING_EVENT_CLASS)
            (x_train, y_train), _ = generate_train_test(preprocessed_data)

            # split_index = int(x_train.shape[0] * .8)
            #
            # x_val, y_val = x_train[split_index:], y_train[split_index:]
            # x_train, y_train = x_train[:split_index], y_train[:split_index]
            #
            # train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).cache().prefetch(tf.data.AUTOTUNE)
            # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).cache().prefetch(tf.data.AUTOTUNE)

            tuner = kt.Hyperband(
                model_builder,
                objective='val_accuracy',
                max_epochs=200,
                factor=3,
                seed=seed,
                directory=os.path.join(file_helper.get_root_path(), f'data/hypermodel/date_o_d/{offset}/{duration}'),
                project_name='geo_analysis'
            )

            tuner.search(
                # train_dataset,
                x_train, y_train,
                epochs=100,
                validation_split=.2,
                # validation_data=val_dataset,
                callbacks=[stop_early]
            )
            score = tuner.oracle.get_best_trials(num_trials=1)[0].score

            if best_score < score:
                best_score = score
                best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
                best_preprocessed = preprocessed_data
                best_reserved = reserved_data
                best_tuner = tuner
                best_offset, best_duration = (offset, duration)
            last_score = score

    model = best_tuner.hypermodel.build(best_hps)

    print(f'{"Best score":10}:{best_score}\n{"Offset":10}:{best_offset}\n{"Duration":10}:{best_duration}')

    return model, best_preprocessed, best_reserved, best_hps

In [20]:
# model, preprocessed_data, reserved_data, best_hps = search_best_model(seed=42)

In [21]:
print(*[f'{k:20}: {v}' for k, v in best_hps.values.items()], sep='\n')

activation          : linear
units               : 24
activation_1        : tanh
units_1             : 56
activation_2        : sigmoid
units_2             : 40
learning_rate       : 0.001
tuner/epochs        : 200
tuner/initial_epoch : 67
tuner/bracket       : 4
tuner/round         : 4
tuner/trial_id      : 0142


In [46]:
# model = Sequential([
#     layers.InputLayer(input_shape=(5,), name='input'),
#     layers.Normalization(axis=1),
#     layers.Dense(120, activation=activations.tanh, name='dense_0'),
#     layers.Dense(72, activation=activations.relu, name='dense_1'),
#     layers.Dense(64, activation=activations.tanh, name='dense_2'),
#     layers.Dense(1, name='output')
# ])
# model.compile(
#     optimizer=optimizers.Adam(name='adam', learning_rate=.001),
#     loss=losses.BinaryCrossentropy(name='binary_crossentropy', from_logits=True),
#     metrics=[metrics.BinaryAccuracy(name='accuracy')]
# )

model = tuner.hypermodel.build(best_hps)

# inputs = keras.Input(shape=(5,))
# normalization = keras.layers.Normalization(axis=1, name='normalization')(inputs)
# outputs = core_model(normalization)
#
# model = keras.Model(inputs, outputs, name='model')
# model.compile(
#     optimizer=optimizers.Adam(name='adam', learning_rate=.001),
#     loss=losses.BinaryCrossentropy(name='binary_crossentropy', from_logits=True),
#     metrics=[metrics.BinaryAccuracy(name='accuracy')]
# )

In [47]:
(x_train, y_train), (x_test, y_test) = generate_train_test(preprocessed_data, train=TRAIN_FRAC)

In [48]:
split_index = int(x_train.shape[0] * .8)

x_trains, y_trains = x_train[:split_index], y_train[:split_index]
x_val, y_val = x_train[split_index:], y_train[split_index:]

BATCH_SIZE = 32
BUFFER_SIZE = 4096

train_dataset = tf.data.Dataset.from_tensor_slices((x_trains, y_trains)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

history = model.fit(
    # x_train, y_train,
    train_dataset,
    epochs=100,
    # epochs=best_hps.get('tuner/epochs'),
    # validation_split=.2,
    validation_data=val_dataset,
    # verbose=0,
    class_weight={0: weight_0, 1: weight_1},
    # callbacks=[stop_early],
)
# len(history.history['val_loss'])

Epoch 1/100
113/113 [==============================] - 3s 17ms/step - loss: 0.7086 - accuracy: 0.8136 - val_loss: 0.6511 - val_accuracy: 0.9610
Epoch 2/100
113/113 [==============================] - 2s 14ms/step - loss: 0.6987 - accuracy: 0.9565 - val_loss: 0.6545 - val_accuracy: 0.9610
Epoch 3/100
113/113 [==============================] - 1s 13ms/step - loss: 0.6847 - accuracy: 0.9234 - val_loss: 0.6413 - val_accuracy: 0.9610
Epoch 4/100
113/113 [==============================] - 1s 13ms/step - loss: 0.6594 - accuracy: 0.8989 - val_loss: 0.6155 - val_accuracy: 0.9555
Epoch 5/100
113/113 [==============================] - 2s 13ms/step - loss: 0.6193 - accuracy: 0.8799 - val_loss: 0.5777 - val_accuracy: 0.9477
Epoch 6/100
113/113 [==============================] - 1s 13ms/step - loss: 0.5682 - accuracy: 0.8529 - val_loss: 0.5309 - val_accuracy: 0.9131
Epoch 7/100
113/113 [==============================] - 1s 12ms/step - loss: 0.5151 - accuracy: 0.8426 - val_loss: 0.4824 - val_accuracy:

In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=history.epoch,
    y=history.history['loss'],
    name='loss',
    line=dict(dash='dash', color='royalblue')
))
fig.add_trace(go.Scatter(
    x=history.epoch,
    y=history.history['val_loss'],
    name='val_loss',
    line=dict(dash='dash', color='firebrick')
))
fig.add_trace(go.Scatter(
    x=history.epoch,
    y=history.history['accuracy'],
    name='accuracy',
    line=dict(color='royalblue')
))
fig.add_trace(go.Scatter(
    x=history.epoch,
    y=history.history['val_accuracy'],
    name='val_accuracy',
    line=dict(color='firebrick')
))
fig.update_layout(
    title='Model losses and accuracies',
    width=600, height=400
)

fig.show()

In [50]:
# model.summary()

In [51]:
# keras.utils.vis_utils.plot_model(model, show_shapes=True)

In [52]:
model.evaluate(test_dataset)

36/36 [==============================] - 0s 13ms/step - loss: 0.1133 - accuracy: 0.9626


[0.11333804577589035, 0.9625668525695801]

In [53]:
y_pred = tf.round(tf.nn.sigmoid(model.predict(x_test)))

tp = keras.metrics.TruePositives(name='tp')
tp.update_state(y_test, y_pred)

fp = keras.metrics.FalsePositives(name='fp')
fp.update_state(y_test, y_pred)

tn = keras.metrics.TrueNegatives(name='tn')
tn.update_state(y_test, y_pred)

fn = keras.metrics.FalseNegatives(name='fn')
fn.update_state(y_test, y_pred)

prec = keras.metrics.Precision(name='precision')
prec.update_state(y_test, y_pred)

recall = keras.metrics.Recall(name='recall')
recall.update_state(y_test, y_pred)

roc = keras.metrics.AUC(curve='ROC', name='roc')
roc.update_state(y_test, y_pred)

prc = keras.metrics.AUC(curve='PR', name='prc')
prc.update_state(y_test, y_pred)

print(f'{"=" * 33}')
print(f'|| {" " * 9} | {"Actual":15} ||')
print(f'|| {"Predicted":9} | {"-" * 15} ||')
print(f'|| {" " * 9} | {"Pos":6} | {"Neg":6} ||')
print(f'|| {"-" * 9} + {"-" * 6} + {"-" * 6} ||')
print(f'|| {"Pos":9} | {tp.result():6} | {fp.result():6} ||')
print(f'|| {"-" * 9} + {"-" * 6} + {"-" * 6} ||')
print(f'|| {"Neg":9} | {fn.result():6} | {tn.result():6} ||')
print(f'{"=" * 33}')

print(f'{"Precision":10}: {prec.result():.5}')
print(f'{"Recall":10}: {recall.result():.5}')
print(f'{"ROC":10}: {roc.result():.5}')
print(f'{"PRC":10}: {prc.result():.5}')

||           | Actual          ||
|| Predicted | --------------- ||
||           | Pos    | Neg    ||
|| --------- + ------ + ------ ||
|| Pos       |   37.0 |   47.0 ||
|| --------- + ------ + ------ ||
|| Neg       |    1.0 | 1037.0 ||
Precision : 0.44048
Recall    : 0.97368
ROC       : 0.96516
PRC       : 0.43134


In [54]:
condition = (convert_string_to_date('1.1.2021') <= reserved_data['to date'])

X_pred = reserved_data[condition].drop(['to date'], axis=1)

# X_pred = processed_data[condition]

result_pred = model.predict(X_pred)
result_proba = tf.nn.sigmoid(result_pred).numpy()

result_proba[:10]

array([[0.00017478],
       [0.00017478],
       [0.00017478],
       [0.00027468],
       [0.00020917],
       [0.00026255],
       [0.00022639],
       [0.00024839],
       [0.00021213],
       [0.00021213]], dtype=float32)

In [55]:
figure_x = reserved_data[condition]['to date']
figure_x_text = figure_x.map(lambda e: f'[{(e + DATE_TARGET_OFFSET).strftime("%d.%m.%Y")}..{(e + DATE_TARGET_OFFSET + DATE_TARGET_DURATION).strftime("%d.%m.%Y")}]')
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=figure_x,
    y=1-result_proba[:,0],
    mode='lines',
    stackgroup='one',
    line=dict(width=.25),
    groupnorm='percent',
    name='No event',
    text=figure_x_text
))
fig.add_trace(go.Scatter(
    x=figure_x,
    y=result_proba[:,0],
    mode='lines',
    stackgroup='one',
    line=dict(width=.25),
    name='Event predicted',
    text=figure_x_text
))
fig.update_layout(
    title='Probability of event',
    # width=500, height=500
)
fig.show()

In [56]:
result = pd.DataFrame(reserved_data[condition])

In [57]:
result['predicted'] = result_pred
result[(result['predicted'] > 0)]['to date'].map(
    lambda date:
    f'K >= {STARTING_EVENT_CLASS}; Dates [{(date + DATE_TARGET_OFFSET).strftime("%d.%m.%Y")}..{(date + DATE_TARGET_OFFSET + DATE_TARGET_DURATION).strftime("%d.%m.%Y")}]'
)

Series([], Name: to date, dtype: object)